In [31]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
from datasets import Dataset
from transformers import BertTokenizer, TFBertForSequenceClassification, pipeline
# from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

ImportError: cannot import name 'isfilelike' from 'fsspec.utils' (/opt/anaconda3/lib/python3.9/site-packages/fsspec/utils.py)

In [16]:
# create a new csv file of only the 'GOING_CONCERN' (labels) and the 'OPINION_TEXT1' columns
import pandas as pd

# Load the CSV file
df = pd.read_csv("data/Audit Analytics 01.2010.csv")

# Select only the columns you need
df = df[['OPINION_TEXT1', 'GOING_CONCERN']]

# Rename the columns
df = df.rename(columns={'OPINION_TEXT1': 'report', 'GOING_CONCERN': 'label'})

# Save the new CSV file
df.to_csv("data/going_concern_only.csv", index=False)


/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (0,18,19,20,21,45,46,47,52,53,63,65,67,75,76,77,78,79,80,81,82,84,85,86,87,88,89,105,106,107,108,124,126,136,144,145,146,147,153,154,155,156,158,159,160,161,176,178,196,197,228,230,235,236,237,238,274,278,279,280,281,293,294,295,296,328,329,330,351,371,372,373,374,376,476,477) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [19]:
# Load the CSV file
df = pd.read_csv("data/going_concern_only.csv")

# Count the number of null or empty values in the label column
null_count = df['label'].isnull().sum()

# Print the result
print("There are", null_count, "null or empty values in the label column.")


There are 0 null or empty values in the label column.


In [18]:
# Drop rows with null or empty values in the going_concern column
df = df.dropna(subset=['label'])

# Save the new CSV file
df.to_csv("data/going_concern_only.csv", index=False)

In [20]:
df.head()

,report,label
0,Report of Independent Registered Public Accoun...,0.0
1,Report of Independent Registered Public Accoun...,0.0
2,Report of Independent Registered Public Accoun...,0.0
3,Report of Independent Registered Public Accoun...,0.0
4,Report of Independent Registered Public Accoun...,0.0


In [21]:
df = df.dropna(subset=['report', 'label'])

In [22]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(134956, 2) (16662, 2) (14996, 2)


In [ ]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [24]:
model = TFBertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain', from_pt=True, num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Loading a PyTorch model in TensorFlow, requires both PyTorch and TensorFlow to be installed. Please see https://pytorch.org/ and https://www.tensorflow.org/install/ for installation instructions.


ModuleNotFoundError: No module named 'torch'

In [10]:
finbert = TFBertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at yiyanghkust/finbert-tone.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [11]:
nlp = pipeline("text-classification", model=finbert, tokenizer=tokenizer)
results = nlp(['growth is strong and we have plenty of liquidity.', 
               'there is a shortage of capital, and we need extra financing.',
              'formulation patents might protect Vasotec to a limited extent.'])

In [12]:
results

[{'label': 'Positive', 'score': 1.0},
 {'label': 'Negative', 'score': 0.9952379465103149},
 {'label': 'Neutral', 'score': 0.9979718327522278}]

In [ ]:
import numpy as np

finbert = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

sentences = ["there is a shortage of capital, and we need extra financing", 
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]

inputs = tokenizer(sentences, return_tensors="pt", padding=True)
outputs = finbert(**inputs)[0]

labels = {0:'neutral', 1:'positive',2:'negative'}
for idx, sent in enumerate(sentences):
    print(sent, '----', labels[np.argmax(outputs.detach().numpy()[idx])])
    
'''
there is a shortage of capital, and we need extra financing ---- negative
growth is strong and we have plenty of liquidity ---- positive
there are doubts about our finances ---- negative
profits are flat ---- neutral
'''